## Amazon DynamoDB Hands-on LAB

본 워크샵을 통해 Amazon DynamoDB를 SageMaker Notebook 또는 SageMaker Studio 환경에서 간단히 실습합니다.

Purpose-built database ['Getting Started' 핸즈온](https://aws.amazon.com/ko/getting-started/hands-on/purpose-built-databases/dynamodb/)을 참고하였습니다. 

## 개요 

- 이 과정에서는 사용자가 레스토랑을 평가하고 리뷰를 남길 수 있는 레스토랑 리뷰 애플리케이션을 만듭니다. 그러면 다른 사용자가 이러한 리뷰를 검색하여 인기 있는 레스토랑을 찾을 수 있습니다.

- 이 단원에서는 레스토랑을 평가하고 리뷰하기 위한 핵심 서비스를 구축합니다. 사용자는 레스토랑을 리뷰하거나 평가할 수 있고, 다른 사용자는 레스토랑을 선택할 때 리뷰와 평점을 찾아볼 수 있습니다.

- 이 서비스에는 AWS의 NoSQL 데이터베이스인 Amazon DynamoDB를 사용합니다. 이 단원에서는 애플리케이션에서 완전히 관리되는 DynamoDB 테이블을 사용하는 방법을 설명합니다. 먼저 DynamoDB를 사용해야 하는 이유를 알아봅니다. 그런 다음 DynamoDB 테이블을 생성하고 애플리케이션에서 사용하는 단계를 살펴봅니다. 이 세션을 마치면 애플리케이션에서 DynamoDB를 사용할 수 있다는 자신감을 갖게 될 것입니다.


## 왜 DynamoDB를 사용할까요?

- DynamoDB는 모든 규모에서 빠르고 일관된 성능을 제공하는 완전 관리형 NoSQL 데이터베이스입니다. 유연한 과금 모델, 코드형 인프라와의 긴밀한 통합, 핸즈오프 운영 모델을 갖추고 있습니다.

- DynamoDB는 두 가지 범주의 애플리케이션에서 선택되는 데이터베이스가 되었습니다:
    - 대규모 애플리케이션: DynamoDB는 확장성을 위해 만들어졌습니다. Amazon 엔지니어들이 글로벌 규모의 트래픽을 처리하기 위해 Amazon.com이라는 리테일 인프라를 확장하면서 얻은 학습을 기반으로 합니다. DynamoDB는 Airbnb, Lyft와 같이 빠르게 성장하는 스타트업부터 Capital One, 삼성과 같은 대기업에 이르기까지 많은 엔지니어링 팀에서 대규모로 사용하고 있습니다. DynamoDB는 규모에 관계없이 동일하고 일관된 성능을 제공할 수 있으므로 데이터가 증가해도 데이터베이스를 리팩터링할 필요가 없습니다.
    - 서버리스 애플리케이션: DynamoDB는 AWS Lambda 및 AWS API 게이트웨이와 같은 서비스를 사용하여 서버리스 애플리케이션을 구축하는 개발자들에게 인기가 있습니다. Lambda의 임시 컴퓨팅 특성은 기존 데이터베이스가 처리하기 어렵습니다. DynamoDB는 서버리스 컴퓨팅과 잘 작동하는 HTTP 연결 모델과 AWS ID 및 액세스 관리(IAM) 인증을 갖추고 있습니다. 또한 DynamoDB는 서버리스 환경에 적합한 사용량 기반 요금제 옵션을 제공합니다.

DynamoDB는 대규모의 서버리스 애플리케이션용으로 인기있는 데이터베이스이지만, 거의 모든 온라인 트랜잭션 처리(OLTP) 애플리케이션 워크로드에서 잘 작동합니다. DynamoDB는 엔티티 간의 관계는 물론 복잡한 필터링 및 정렬 요구 사항도 처리할 수 있습니다.

## 단원 내용

### 1. SageMaker Notebook 환경 구성하기

이 단원에서는 Python을 사용하여 DynamoDB 테이블과 상호 작용합니다. SageMaker Notebook에서 다음 명령을 실행하여 모듈 코드를 다운로드하고 압축을 풉니다.


In [1]:
!curl -sL https://s3.amazonaws.com/aws-data-labs/restaurant-dynamodb.tar | tar -xv

scripts/
scripts/bulk_load_table.py
scripts/items.json
scripts/delete_table.py
scripts/create_table.py
application/
application/create_restaurant.py
application/create_review.py
application/entities.py
application/fetch_restaurant_summary.py
requirements.txt


SageMaker Notebook에 두 개의 디렉터리가 표시되어야 합니다:

- scripts/: 이 디렉터리에는 테이블 생성, 보조 인덱스 추가, 완료 시 테이블 삭제 등 DynamoDB 테이블을 조작하기 위한 관리 스크립트가 포함되어 있습니다.
- application/: 이 디렉토리에는 DynamoDB 테이블의 데이터와 상호 작용하기 위한 코드가 포함되어 있습니다. 어플리케이션에 있는 코드와 유사합니다.

다음 명령을 실행하여 종속성 있는 라이브러리를 설치합니다.

In [15]:
!pip install -r ../requirements.txt

  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: redis
    Found existing installation: redis 3.5.2
    Uninstalling redis-3.5.2:
      Successfully uninstalled redis-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [12]:
!pip install ksuid

  Using cached ksuid-1.3.tar.gz (2.2 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for ksuid: filename=ksuid-1.3-py3-none-any.whl size=3276 sha256=817dfe417abd88c1a7798958ca531170dba8bf37c28af9c964ab02d20e31aaf2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/eb/81/22/b0abee501706f015532a5d0d1c303926eebd211a08a6dfdbd7
Successfully built ksuid


In [13]:
!pip freeze > requirements2.txt

이 모듈에서는 개발에 사용할 SageMaker Notebook을 구성했습니다. 다음 모듈에서는 DynamoDB를 사용하여 데이터 모델을 설계합니다.

### 2. DynamoDB용 데이터 모델링 준비하기

특정 데이터베이스로 작업할 때는 해당 데이터베이스 스타일에 맞는 방식으로 데이터를 모델링해야 합니다. 데이터 모델링 스타일은 데이터베이스마다 다릅니다.

관계형 데이터베이스에서는 엔티티를 원자 단위로 정규화하여 각 엔티티를 별도의 테이블에 표시합니다. 그런 다음 테이블 간에 참조를 사용하여 엔티티 간의 관계를 나타냅니다. 데이터를 정규화한 후에는 원하는 데이터를 가져오는 SQL 쿼리를 작성하여 액세스 패턴을 처리합니다. 이러한 방식으로, 액세스 패턴을 처리하기 전에 데이터를 설계합니다.

NoSQL 데이터베이스를 사용하면 그 반대입니다. 먼저 액세스 패턴을 고려한 다음 해당 액세스 패턴을 처리하도록 데이터 모델을 설계합니다. 액세스 패턴에 맞게 특별히 설계하면 관계형 데이터베이스보다 더 확장할 수 있는 효율적인 데이터베이스를 만들 수 있습니다.

관계형 데이터베이스에 익숙한 경우에는, DynamoDB와 같은 NoSQL 데이터베이스를 사용한 데이터 모델링이 낯설게 보일 수 있습니다. DynamoDB에 맞는 데이터 모델링의 원칙을 준수해야 합니다.

DynamoDB와 같은 NoSQL 데이터베이스로 데이터를 모델링할 때는 다음 세 단계를 따르세요:

  1. 엔티티 관계 다이어그램(ERD)을 만듭니다. ERD는 애플리케이션의 다양한 개체(또는 엔티티)를 나열하고 관계를 통해 서로 어떻게 연관되어 있는지 보여줍니다. 이 다이어그램은 애플리케이션의 핵심 개념을 이해하는 데 도움이 됩니다.
  2. 액세스 패턴을 이해하세요. ERD를 만든 후에는 애플리케이션에서 개체에 액세스하는 다양한 방법을 나열하세요. 여기에는 읽기 기반 및 쓰기 기반 액세스 패턴이 모두 포함됩니다.
  3. 액세스 패턴을 처리할 수 있도록 데이터 모델을 설계합니다. 액세스 패턴을 나열한 후에는 특정 액세스 패턴을 처리하도록 데이터 모델을 설계하세요.

DynamoDB를 사용한 NoSQL 데이터 모델링은 광범위한 주제이며, 이 강의에서는 모든 원칙을 다루지는 않습니다. DynamoDB를 사용한 NoSQL 데이터 모델링에 대한 자세한 내용은 [DynamoDB를 사용한 설계 및 아키텍처 모범 사례에 대한 DynamoDB 설명서](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/best-practices.html)를 참조하세요. 또한 [DynamoDB를 사용한 데이터 모델링의 다른 실습 예제](https://aws.amazon.com/getting-started/hands-on/?getting-started-all.q=dynamodb&getting-started-all.q_operator=AND)도 확인할 수 있습니다.

이러한 원칙을 염두에 두고 데이터 모델 준비를 시작해 보겠습니다.

### (1) 엔티티-관계 다이어그램 만들기

이 서비스에서는 레스토랑 리뷰를 처리합니다. 사용자는 애플리케이션에서 계정을 생성하고 방문한 레스토랑에 대한 리뷰를 남길 수 있습니다. 리뷰에는 평점(1~5점)과 사용자의 방문에 대한 텍스트 기반 요약이 모두 포함됩니다. 이 애플리케이션을 통해 사용자는 레스토랑 세부 정보, 평점 요약, 가장 최근 리뷰 등 레스토랑에 대한 요약을 볼 수 있습니다. 또한 이 애플리케이션을 통해 사용자는 우편번호로 레스토랑을 검색할 수 있습니다.

이 서비스에서 사용자를 처리하는 것은 이 과정의 뒷부분에 있는 [Amazon Keyspaces 단원](https://aws.amazon.com/ko/keyspaces/)에서 다루기 때문에 여기서는 다루지 않습니다.

이를 염두에 두고 ERD를 살펴보겠습니다.

<img src="img/p10.png" width="600" height="200">

ERD는 매우 간단하며 두 개의 엔티티만 포함합니다: 레스토랑과 리뷰입니다. 레스토랑은 다양한 사용자로부터 리뷰를 받을 수 있으므로 레스토랑과 리뷰 사이에는 일대다 관계가 있습니다.

이제 엔티티와 관계를 알았으므로 다음 단계로 넘어갈 수 있습니다.

### (2) 애플리케이션 액세스 패턴 목록 만들기
다음 단계는 애플리케이션의 액세스 패턴을 나열하는 것입니다. 효과적인 테이블을 설계할 수 있도록 모든 액세스 패턴을 사전에 철저히 고려하는 것이 중요합니다.

가장 중요한 액세스 패턴은 **Fetch Restaurant Summary**입니다. 사용자가 레스토랑 페이지로 이동하면 레스토랑 세부 정보, 모든 평점 요약, 가장 최근 리뷰 5개 등 레스토랑에 대한 가장 관련성 높은 정보를 보고 싶어 합니다.

이 액세스 패턴의 일부로 두 가지 핵심 엔티티에 대한 기본적인 만들기 및 읽기 액세스 패턴이 있습니다. 또한 **Create Review** 액세스 패턴의 경우, 제출하는 사용자가 해당 레스토랑을 이미 리뷰하지 않았는지 확인하기 위해 고유성(Uniqueness)을 적용해야 합니다. 이렇게 하면 한 사용자가 부정적 또는 긍정적 리뷰를 여러 개 제출하여 레스토랑의 평점을 왜곡하는 것을 방지할 수 있습니다.

마지막으로, 여러 가지 이유로 리뷰를 삭제해야 하는 경우가 있을 수 있습니다. 리뷰가 불필요하게 악의적이거나 레스토랑과 관련된 누군가가 전체 평점을 부풀리기 위해 리뷰를 게시했기 때문일 수 있습니다. 따라서 리뷰 삭제 액세스 패턴이 있습니다.

애플리케이션에서 처리해야 하는 액세스 패턴은 총 6가지입니다:

- Fetch Restaurant Summary
- Create Restaurant
- Get Restaurant
- Create Review
- Get Review
- Remove Review

이제 애플리케이션에 대한 액세스 패턴을 확보했으므로 이러한 액세스 패턴을 처리하도록 테이블을 디자인하는 작업을 진행합니다.

이 모듈에서는 DynamoDB로 데이터 모델을 준비하는 방법에 대해 배웠습니다. 먼저, 관계형 데이터베이스를 사용한 데이터 모델링 프로세스와 DynamoDB를 사용한 데이터 모델링 프로세스가 어떻게 다른지 배웠습니다. 그런 다음 레스토랑 평가 애플리케이션에 대한 엔티티-관계 다이어그램을 작성했습니다. 마지막으로 애플리케이션의 액세스 패턴을 나열했습니다.

다음 모듈에서는 기본 키(Primary key)와 보조 인덱스(Secondary index)를 설계하여 데이터 모델을 계획하는 방법을 배웁니다.

### 3. 데이터 모델을 계획하고 샘플 데이터를 로드하기

이 모듈에서는 액세스 패턴을 처리하기 위한 DynamoDB 데이터 모델을 계획하기 시작합니다. 이 데이터 모델을 계획하면서 DynamoDB를 사용한 데이터 모델링에 대한 몇 가지 주요 원칙을 배우게 됩니다.

이전 모듈에서는 DynamoDB에서 데이터를 모델링하는 프로세스가 관계형 데이터베이스에서 데이터를 모델링하는 프로세스와 다르다는 것을 배웠습니다. 프로세스의 차이 외에도 DynamoDB에서 데이터를 모델링할 때 적용되는 원칙이 다릅니다.

DynamoDB 데이터 모델링의 원칙을 이해하려면 DynamoDB에 대한 두 가지 설계 특징을 이해해야 합니다.

- 첫째, DynamoDB는 테이블 간 조인을 허용하지 않습니다. 조인은 쿼리 시 여러 테이블의 값을 결합하여 최종 결과에 도달하는 관계형 데이터베이스의 일반적인 기능입니다. 데이터가 확장됨에 따라 조인은 여러 테이블에서 큰 값 덩어리를 읽어야 하므로 점점 더 느리게 수행됩니다. 이러한 확장 문제를 방지하기 위해 DynamoDB는 조인을 완전히 제거했습니다.

- 둘째, 데이터를 쿼리하는 데 사용할 수 있는 속성이 제한되어 있습니다. 관계형 데이터베이스에서는 쿼리에서 테이블의 모든 열을 사용할 수 있습니다. DynamoDB에서는 [기본 키](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/HowItWorks.CoreComponents.html#HowItWorks.CoreComponents.PrimaryKey)를 사용하여 데이터에 액세스하는 데 중점을 둡니다. 기본 키를 올바르게 모델링하는 것은 효과적인 DynamoDB 데이터 모델을 위해 매우 중요합니다.

이 두 가지 설계 특징은 DynamoDB 테이블이 어떤 크기로 확장되더라도 일관된 성능을 유지하는 데 도움이 됩니다. 이 두 가지 설계 특징에 따라 데이터를 모델링할 때 다음 원칙을 사용해야 합니다:

- **단일 테이블에 여러 엔티티를 넣습니다.** 앞서 언급했듯이, DynamoDB는 테이블 간 조인을 허용하지 않습니다. 조인과 유사한 기능을 허용하려면 단일 테이블에 서로 다른 여러 엔터티 유형을 넣어야 합니다. 이 예에서는 **레스토랑**과 **리뷰**가 같은 테이블 내에 있습니다.
- **기본 키에 일반 이름과 값을 사용합니다.** 모든 데이터 액세스는 기본 키를 통해 이루어집니다. 그러나 DynamoDB 테이블에는 서로 다른 여러 엔티티가 있고, 일부 엔티티에는 이러한 속성이 없으므로 RestaurantName 또는 RatingId와 같은 기본 키 이름을 사용할 수 없습니다. 대신 **PK**(파티션 키의 경우) 및 **SK**(정렬 키의 경우)와 같은 이름을 사용합니다. 기본 키의 값은 액세스 패턴에 도움이 되도록 특별히 설계되었습니다.
- **추가 액세스 패턴을 처리하려면 보조 인덱스를 사용합니다.** 테이블의 기본 키는 개별 항목의 생성 및 삭제와 관련된 기본 액세스 패턴을 처리합니다. 한 번의 요청으로 여러 항목을 가져와야 하는 고급 액세스 패턴도 처리할 수 있습니다. 그러나 테이블의 기본 키 디자인에 맞지 않는 추가 액세스 패턴이 있는 경우가 있습니다. [보조 인덱스](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/SecondaryIndexes.html)를 사용하여 이러한 추가 패턴을 활성화할 수 있습니다. 보조 인덱스는 테이블에서 추가 기본 키처럼 작동하며 보조 액세스 패턴에 빠르게 액세스할 수 있게 해줍니다. 테이블에 항목을 삽입하면 DynamoDB는 새로운 기본 키 패턴을 사용하여 해당 항목을 보조 인덱스에 복사하는 작업을 처리합니다.

이러한 원칙을 염두에 두고 액세스 패턴을 처리하는 기본 키를 설계해 보겠습니다.


### (1) 기본 키 설계하기

기본 키 설계는 DynamoDB 데이터 모델링에서 매우 중요한 부분입니다. 기본 키는 테이블에서 데이터를 식별하고 액세스하는 방법을 결정합니다.

첫 번째 단계는 원하는 기본 키 유형을 결정하는 것입니다. DynamoDB에는 두 가지 유형의 기본 키가 있습니다. 첫 번째 유형은 단순 기본 키이며 하나의 요소, 즉 파티션 키로 구성됩니다. 두 번째 유형은 복합 기본 키이며 파티션 키와 정렬 키의 두 가지 요소로 구성됩니다.

단순 기본 키를 사용하면 DynamoDB 항목에 대해 개별 키-값 작업만 수행할 수 있습니다. 복합 기본 키를 사용하면 쿼리 작업을 사용하여 동일한 파티션 키를 가진 여러 항목을 검색하는 등 보다 고급 패턴을 수행할 수 있습니다.

대부분의 애플리케이션에서 복합 기본 키를 선택하는 이유는 애플리케이션이 성장함에 따라 더 복잡한 모델링과 추가적인 유연성을 허용하기 때문입니다. 이 단원에서는 복합 기본 키를 사용합니다.

기본 키 유형을 선택한 후에는 엔티티에 대한 기본 키를 디자인해야 합니다. 가장 먼저 고려해야 할 것은 고유성 요구 사항입니다. 엔티티가 특정 차원에서 고유해야 하는 경우 해당 요구 사항을 기본 키에 적용해야 합니다.

이 단원에서는 두 엔터티에 대한 고유성 요구 사항이 있습니다. 첫째, **Restaurant**은 그 이름으로 고유하게 식별되어야 합니다. 둘째, 사용자가 하나의 레스토랑에 대해 여러 개의 리뷰를 남기지 못하도록 해야 합니다. 따라서 **Review** 항목은 레스토랑 이름과 사용자 이름의 조합으로 고유하게 식별되어야 합니다.

이를 염두에 두고 이 두 엔티티에 다음과 같은 기본 키 구조를 사용할 수 있습니다.

<img src="img/p1.png" width="900" height="400">

**Restaurant** 엔터티의 경우 **PK**와 **SK**의 값은 모두 REST#<RestaurantName>입니다.

**Review** 엔터티의 경우 **PK**의 값은 USER#<UserName>이고 **SK**의 값은 REST#<RestaurantName>입니다. 사용자 이름과 레스토랑 이름을 모두 기본 키로 인코딩하면 두 속성의 조합이 특정 리뷰에 대해 고유하다고 할 수 있습니다.

두 엔티티 유형 모두에 대해 기본 키 값에 접두사(REST# 및 USER#)를 포함하고 있는 것을 알 수 있습니다. 이렇게 하면 항목 유형을 식별하고 엔티티 유형 간의 충돌을 방지하는 데 도움이 됩니다.

데이터 모델링을 돕기 위해 DynamoDB용 NoSQL Workbench를 사용할 수 있습니다. 이 도구를 사용하면 테이블을 만들고 항목을 삽입하고 테이블에서 해당 항목이 어떻게 보이는지 확인할 수 있습니다.

다음은 NoSQL Workbench에 로드된 일부 **Restaurant** 및 **Review** 항목의 예입니다.
    
<img src="img/p2.png" width="900" height="600">

처음 두 항목은 **Susan's Steaks**와 **Thai Time** 레스토랑에 대한 리뷰입니다. 다음 두 항목은 **Thai Time** 레스토랑에 대한 리뷰입니다.

이 기본 기본 키 패턴은 이미 6개의 액세스 패턴 중 5개를 충족했습니다:

- Create Restaurant
- Get Restaurant
- Create Review
- Get Review
- Remove Review

 
다음 모듈에서는 이러한 액세스 패턴 중 일부를 처리하는 몇 가지 코드를 살펴보겠습니다. 다음으로, 좀 더 복잡한 액세스 패턴인 **Fetch Restaurant Summary** 액세스 패턴을 처리하는 방법을 살펴보겠습니다.

### (2) 비정규화 및 보조 인덱스 사용하기

우리가 처리하고자 하는 최종 액세스 패턴은 더 복잡합니다. 다음을 포함하는 **Fetch Restaurant Summary**입니다:

- 레스토랑 세부 정보가 포함된 레스토랑 항목.
- 레스토랑에 대한 모든 리뷰의 평점(1~5점) 요약.
- 레스토랑에 대한 가장 최근 5개 리뷰의 텍스트.

이 액세스 패턴에 도움이 되는 두 가지 전략을 사용할 것입니다.

첫째, 사용자가 레스토랑을 볼 때마다 평점 요약을 다시 계산하기 위해 레스토랑에 대한 모든 **Review** 항목을 가져오는 것은 비효율적입니다. 대신 레스토랑에 대한 모든 평점 요약을 저장하여 데이터를 비정규화합니다. 각 레스토랑 항목에는 레스토랑이 받은 별점 1점, 별점 2점, 별점 3점, 별점 4점 및 별점 5점의 수를 나타내는 5개의 속성이 있습니다. 새 리뷰가 생성될 때마다 애플리케이션은 상위 레스토랑 항목의 관련 카운터 속성도 증가시킵니다. 이를 통해 평균 평점, 평점 수 및 평점 분포를 빠르게 표시할 수 있습니다.

이제 NoSQL 워크벤치에서 항목은 다음과 같이 표시됩니다.

<img src="img/p4.png" width="900" height="600">

이제 두 개의 **Restaurant** 항목에 별 개수별 평점 수에 대한 속성이 있는 것을 확인할 수 있습니다.

두 번째 전략은 한 번의 요청으로 **Restaurant** 항목과 가장 최근의 **Review** 항목 5개를 검색하는 데 도움이 됩니다. 이를 처리하려면 보조 인덱스를 사용하여 데이터를 재구성합니다. 보조 인덱스는 추가 쿼리 패턴을 허용하기 위해 테이블에 기본 키 구조를 추가합니다.

보조 인덱스의 키 구조는 다음과 같습니다.

<img src="img/p5.png" width="900" height="600">

이제 항목에 두 개의 새 속성(**GSI1PK** 및 **GSI1SK**)을 추가합니다. 이러한 속성은 기본 키와 마찬가지로 일반 키 이름입니다.

**Restaurant**과 **Review** 항목 모두 **GSI1PK**의 값이 동일하다는 것을 알 수 있습니다. 항목의 파티션 키 값이 같으면 동일한 *항목 컬렉션*에 속한다고 합니다. **쿼리** 작업을 사용하면 한 번의 요청으로 동일한 항목 컬렉션에 있는 여러 항목을 검색할 수 있습니다.

**Review**가 생성될 때마다 애플리케이션은 **GSI1SK** 값에 사용되는 **ReviewId**를 할당합니다. **ReviewId**는 [K-Sortable Unique IDentifer(KSUID)](https://github.com/segmentio/ksuid)입니다. KSUID는 UUID(범용 고유 식별자)의 일부 고유성 보장을 제공하는 동시에 시간 기반 접두사를 포함하여 시간순으로 정렬할 수 있도록 합니다. 이를 통해 특정 레스토랑에 대한 가장 최근의 **Review** 항목을 가져올 수 있습니다.

이제 NoSQL Workbench의 기본 테이블은 다음과 같이 표시됩니다.

<img src="img/p6.png" width="900" height="600">

모든 항목(빨간색으로 윤곽선 표시)에 대해 **GSI1PK** 및 **GSI1SK** 값이 추가된 것을 확인할 수 있습니다. 이전 단계에서 추가되었던 등급은 보기 쉽도록 제거되었습니다.

**GSI1** 보조 인덱스를 보도록 전환하면 보조 인덱스에 대해 데이터가 어떻게 재배열되는지 확인할 수 있습니다.

<img src="img/p7.png" width="900" height="600">

**Thai Time Restaurant**항목과 레스토랑에 대한 두 개의 **Review** 항목이 모두 동일한 항목 컬렉션(빨간색으로 윤곽선 표시)에 있는 것을 확인할 수 있습니다.

다음 모듈에서는 이 데이터를 검색하여 액세스 패턴을 처리하는 방법을 배웁니다.


### (3) DynamoDB 테이블 생성하기

다운로드한 scripts/ 디렉토리에 애플리케이션에 대한 DynamoDB 테이블을 생성하는 create_table.py라는 파일이 있습니다. 파일의 내용은 다음과 같습니다.

In [2]:
%%writefile scripts/create_table.py

import boto3

dynamodb = boto3.client("dynamodb")

try:
    dynamodb.create_table(
        TableName="Restaurants",
        AttributeDefinitions=[
            {"AttributeName": "PK", "AttributeType": "S"},
            {"AttributeName": "SK", "AttributeType": "S"},
            {"AttributeName": "GSI1PK", "AttributeType": "S"},
            {"AttributeName": "GSI1SK", "AttributeType": "S"},
        ],
        KeySchema=[
            {"AttributeName": "PK", "KeyType": "HASH"},
            {"AttributeName": "SK", "KeyType": "RANGE"},
        ],
        GlobalSecondaryIndexes=[
            {
                "IndexName": "GSI1",
                "KeySchema": [
                    {"AttributeName": "GSI1PK", "KeyType": "HASH"},
                    {"AttributeName": "GSI1SK", "KeyType": "RANGE"},
                ],
                "Projection": {"ProjectionType": "ALL"},
                "ProvisionedThroughput": {
                    "ReadCapacityUnits": 5,
                    "WriteCapacityUnits": 5,
                },
            }
        ],
        ProvisionedThroughput={"ReadCapacityUnits": 5, "WriteCapacityUnits": 5},
    )
    print("Table created successfully.")
except Exception as e:
    print("Could not create table. Error:")
    print(e)


Overwriting scripts/create_table.py


이 스크립트는 Python용 AWS SDK인 [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)를 사용하여 DynamoDB와 상호 작용합니다. 이 스크립트는 레스토랑 평가 애플리케이션에 사용할 **Restaurants**라는 테이블을 생성합니다. 이 스크립트는 앞서 언급한 일반 이름(**PK** 및 **SK**)을 사용하여 기본 키 요소를 지정합니다. 또한 스크립트는 앞서 설명한 일반 이름을 사용하여 **GSI1**이라는 글로벌 보조 인덱스를 생성합니다. 마지막으로, 스크립트는 테이블에 프로비저닝할 [읽기 용량 단위와 쓰기 용량 단위](https://docs.aws.amazon.com/ko_kr/amazondynamodb/latest/developerguide/HowItWorks.ReadWriteCapacityMode.html#HowItWorks.ProvisionedThroughput.Manual)를 지정합니다.

다음 명령을 실행하여 스크립트를 실행하고 테이블을 생성합니다.

In [4]:
!python scripts/create_table.py

Table created successfully.


테이블이 성공적으로 생성되었다는 메시지가 표시되어야 합니다.

그런 다음 테이블에 샘플 데이터를 로드합니다. 다음 명령을 실행하여 대량 로드 스크립트를 실행합니다.

In [5]:
%%writefile scripts/bulk_load_table.py

import json

import boto3

dynamodb = boto3.resource("dynamodb")
table = dynamodb.Table("Restaurants")

items = []

with open("scripts/items.json", "r") as f:
    for row in f:
        items.append(json.loads(row))

with table.batch_writer() as batch:
    for item in items:
        batch.put_item(Item=item)

print("Items loaded successfully.")


Overwriting scripts/bulk_load_table.py


In [6]:
!python scripts/bulk_load_table.py

Items loaded successfully.


데이터가 성공적으로 로드되었다는 메시지가 표시되어야 합니다.

이 모듈에서는 DynamoDB 데이터 모델을 계획하고 몇 가지 샘플 데이터를 로드했습니다. 이를 위해 몇 가지 주요 DynamoDB 데이터 모델링 원칙에 대해 배웠습니다. 또한 기본 키 및 보조 인덱스와 같은 DynamoDB 개념에 대해 배웠으며, DynamoDB용 NoSQL Workbench를 사용하여 데이터를 모델링하는 방법을 살펴봤습니다. 마지막으로, Python용 AWS SDK인 Boto3를 사용하여 테이블을 생성하고 샘플 데이터를 로드했습니다.

다음 모듈에서는 애플리케이션 코드에서 DynamoDB와 상호 작용하는 방법을 살펴봅니다.

### 4. 애플리케이션에서 DynamoDB와 상호 작용하기

이 모듈에서는 애플리케이션에서 데이터베이스로서 DynamoDB와 상호 작용하는 방법을 배웁니다. 이전 모듈에서 샘플 데이터로 생성하고 로드한 테이블을 사용합니다.

레스토랑 애플리케이션에는 6가지 액세스 패턴이 있다는 것을 기억하세요:

- Fetch Restaurant Summary
- Create Restaurant
- Get Restaurant
- Create Review
- Get Review
- Remove Review

이 모듈에서는 이러한 액세스 패턴 중 세 가지를 구현하는 코드를 작성하는 방법을 배웁니다. 간단한 예제부터 시작하여 다른 액세스 패턴으로 넘어가면서 더 복잡한 코드를 추가해 보겠습니다.

### 4-1. 레스토랑 생성하기

첫 번째 액세스 패턴은 **Create Restaurant** 액세스 패턴입니다. 이 패턴은 단일 항목에 대한 작업을 포함하므로 가장 간단한 액세스 패턴입니다. 레스토랑 항목을 생성할 때 같은 이름의 기존 레스토랑이 없는지 확인해야 합니다.

**application**/ 디렉터리에 **create_restaurant.py**라는 파일이 있습니다. 내용은 다음과 같습니다.


In [8]:
%%writefile application/create_restaurant.py
import boto3

dynamodb = boto3.client("dynamodb")


def create_restaurant(name, cuisine, address):
    try:
        resp = dynamodb.put_item(
            TableName="Restaurants",
            Item={
                "PK": {"S": "REST#{}".format(name)},
                "SK": {"S": "REST#{}".format(name)},
                "GSI1PK": {"S": "REST#{}".format(name)},
                "GSI1SK": {"S": "REST#{}".format(name)},
                "name": {"S": name},
                "cuisine": {"S": cuisine},
                "address": {"S": address},
            },
            ConditionExpression="attribute_not_exists(PK)",
        )
        return {"name": name, "cuisine": cuisine, "address": address}
    except Exception as e:
        print("Could not create restaurant")
        print(e)
        return False


restaurant = create_restaurant(
    "Bev's Bistro", "French", "541 Salazar Ranch, South Kristen, MS 00857"
)

if restaurant:
    print("Created restaurant: {}".format(restaurant["name"]))

Overwriting application/create_restaurant.py


Boto3 라이브러리를 가져오고 클라이언트를 초기화하면 **create_restaurant**이라는 함수가 있습니다. 이 함수는 애플리케이션 코드에 포함될 함수와 유사합니다. 이 함수는 이름, 요리, 주소를 받고 [PutItem 연산](https://docs.aws.amazon.com/ko_kr/amazondynamodb/latest/APIReference/API_PutItem.html)을 호출하여 테이블에 **Restaurant** 항목을 삽입합니다.

**PutItem 연산**에는 [ConditionExpression](https://docs.aws.amazon.com/ko_kr/amazondynamodb/latest/developerguide/Expressions.ConditionExpressions.html) 매개 변수가 포함되어 있습니다. 이 매개 변수는 동일한 기본 키를 가진 항목이 이미 존재하지 않는지 확인하는 데 사용됩니다. 존재하는 경우 **PutItem** 작업이 거부되고 항목이 기록되지 않습니다.

파일 맨 아래에는 함수 사용 방법에 대한 예제가 있습니다. 이 예제에서는 **create_restaurant** 함수를 호출하여 **Bev's Bistro**라는 새 레스토랑을 생성합니다.

아래 다음 명령을 실행하여 스크립트를 실행하고 새 레스토랑을 생성할 수 있습니다.

In [9]:
!python application/create_restaurant.py

Created restaurant: Bev's Bistro


새 레스토랑 항목이 성공적으로 생성되었다는 메시지가 표시되어야 합니다.

스크립트를 다시 실행해 보세요. 조건부 확인에 실패하여 레스토랑을 만들 수 없다는 오류가 표시될 것입니다. 이는 기존 항목을 실수로 덮어쓰는 것을 방지하기 위해 호출에 포함시킨 **ConditionExpression** 때문입니다.

### 4-2. 레스토랑에 리뷰를 추가하기

다음으로 사용자가 레스토랑을 리뷰할 수 있도록 하는 프로세스를 살펴보겠습니다. 이전 모듈에서 **Restaurant** 항목에 대한 평점을 요약할 수 있도록 DynamoDB 테이블에서 일부 비정규화를 수행한 것을 기억하실 것입니다. 따라서 이 액세스 패턴을 처리할 때는 두 가지 작업을 수행해야 합니다:

- 리뷰 사용자가 아직 이 레스토랑을 리뷰하지 않은 경우 테이블에 **Review** 항목을 추가합니다.
- 상위 **Restaurant** 항목의 관련 평점 속성을 높입니다.

이 두 작업은 모두 함께 성공하거나 실패해야 합니다. 사용자가 이미 이 레스토랑을 리뷰한 경우 상위 **Restaurant**의 평점 속성을 증가시키지 않으려는 것입니다. 또한 평점 속성을 늘리기 위한 쓰기가 실패하면 잘못된 데이터가 생성되므로 리뷰 항목을 저장하고 싶지 않을 것입니다.

이 문제를 처리하기 위해 [DynamoDB 트랜잭션](https://docs.aws.amazon.com/ko_kr/amazondynamodb/latest/developerguide/transactions.html)을 사용할 수 있습니다. 트랜잭션을 사용하면 단일 요청에 여러 작업을 결합하여 전체 요청이 함께 성공하거나 실패하도록 할 수 있습니다. 이를 통해 여러 항목에서 작동하는 복잡한 워크플로를 크게 단순화할 수 있습니다.

**applications**/ 디렉터리에 **create_review.py** 파일이 있습니다.내용은 다음과 같아야 합니다.

In [10]:
%%writefile application/create_review.py

import datetime

import boto3
from ksuid import ksuid

dynamodb = boto3.client("dynamodb")

RATINGS = {
    1: "one_stars",
    2: "two_stars",
    3: "three_stars",
    4: "four_stars",
    5: "five_stars",
}


def create_review(restaurant, username, rating, review_text):
    review_id = str(ksuid())
    rating_attr = RATINGS[rating]
    try:
        resp = dynamodb.transact_write_items(
            TransactItems=[
                {
                    "Put": {
                        "TableName": "Restaurants",
                        "Item": {
                            "PK": {"S": "USER#{}".format(username)},
                            "SK": {"S": "REST#{}".format(restaurant)},
                            "GSI1PK": {"S": "REST#{}".format(restaurant)},
                            "GSI1SK": {"S": "REVIEW#{}".format(review_id)},
                            "username": {"S": username},
                            "restaurant": {"S": restaurant},
                            "id": {"S": review_id},
                            "rating": {"N": str(rating)},
                            "review": {"S": review_text},
                            "created_at": {"S": datetime.datetime.now().isoformat()},
                        },
                        "ConditionExpression": "attribute_not_exists(PK)",
                    },
                },
                {
                    "Update": {
                        "TableName": "Restaurants",
                        "Key": {
                            "PK": {"S": "REST#{}".format(restaurant)},
                            "SK": {"S": "REST#{}".format(restaurant)},
                        },
                        "ConditionExpression": "attribute_exists(PK)",
                        "UpdateExpression": "SET #rating = if_not_exists(#rating, :zero) + :inc",
                        "ExpressionAttributeNames": {"#rating": rating_attr},
                        "ExpressionAttributeValues": {
                            ":inc": {"N": "1"},
                            ":zero": {"N": "0"},
                        },
                    }
                },
            ]
        )
        print("Added review from {} to restaurant {}".format(username, restaurant))
        return True
    except Exception as e:
        print("Could not add review to restaurant")


create_review("Bev's Bistro", "hungryhank", 5, "Great food, great price!")

Overwriting application/create_review.py


여기 구조는 이전 예제와 유사합니다. 필요한 라이브러리를 가져온 후 애플리케이션 코드에 있는 것과 유사한 **create_review** 함수가 있습니다. 이 함수에서는 DynamoDB의 [TransactWriteItems](https://docs.aws.amazon.com/ko_kr/amazondynamodb/latest/APIReference/API_TransactWriteItems.html) 작업을 호출합니다. 트랜잭션에는 두 가지 작업이 포함됩니다:

- 동일한 기본 키를 가진 항목이 존재하지 않는다는 조건으로 **Review** 항목을 생성하는 **PutItem** 작업.
- 상위 **레스토랑** 항목의 관련 평점 속성을 증가시키는 **UpdateItem** 작업.

파일 하단에는 몇 가지 샘플 데이터를 사용하여 **create_review** 함수를 테스트하는 문이 있습니다. 다음 명령을 실행하여 **create_review.py** 스크립트를 실행하고 리뷰를 생성합니다.

In [14]:
!python application/create_review.py

Added review from hungryhank to restaurant Bev's Bistro


사용자 **hungryhank**의 리뷰가 **Bev's Bistro**에 추가되었다는 메시지가 표시되어야 합니다.

스크립트를 다시 실행해 보세요. 이번에는 리뷰를 추가할 수 없다는 오류 메시지가 표시됩니다. 이는 **hungryhank** 사용자가 이미 **Bev's Bistro**에 리뷰를 추가했기 때문입니다.

### 4-3. 레스토랑 요약 불러오기

마지막으로 다룰 액세스 패턴은 **Fetch Restaurant Summary** 액세스 패턴입니다.

사용자가 애플리케이션에서 레스토랑 페이지로 이동하면 레스토랑 자체에 대한 세부 정보뿐만 아니라 받은 모든 평점의 요약과 가장 최근 리뷰 5개의 텍스트를 표시하고 싶습니다.

**Create Rating** 액세스 패턴에서 데이터를 비정규화하고 평점 정보를 **Restaurant** 항목에 복사하여 요약을 빠르게 계산하는 방법을 살펴보았습니다. 이제 한 번의 요청으로 **Restaurant** 항목과 해당 레스토랑의 가장 최근 **Review** 항목 5개를 검색하는 방법을 살펴보겠습니다.

**쿼리** 작업을 사용하면 동일한 파티션 키로 여러 항목을 읽을 수 있습니다. 단일 파티션 키 내에서 항목은 정렬 키 값에 따라 정렬됩니다.

NoSQL 워크벤치에서 일부 **Review** 항목이 있는 **Restaurant** 항목을 모델링한 것을 기억해 보십시오. **GSI1** 보조 인덱스에 대한 뷰는 다음과 같습니다.

<img src="img/p8.png" width="900" height="600">

모든 **Review** 항목이 항목 컬렉션에서 가장 먼저 표시되고 그다음에 **Restaurant** 항목이 표시됩니다. 또한 **Review** 항목은 **ReviewId**에 따라 정렬됩니다. KSUID를 사용하기 때문에 **Review**는 시간순으로 정렬됩니다. 따라서 **Restaurant**과 가장 최근 **Review** 5개를 가져오려면 항목 컬렉션의 끝에서 시작하여 6개의 항목을 검색해야 합니다.

**applications**/ 디렉터리에 **fetch_restaurant_summary.py**라는 스크립트가 있습니다. 내용은 다음과 같아야 합니다.



In [16]:
%%writefile application/fetch_restaurant_summary.py

import boto3

from entities import Restaurant, Review

dynamodb = boto3.client("dynamodb")


def fetch_restaurant_summary(restaurant_name):
    resp = dynamodb.query(
        TableName="Restaurants",
        IndexName="GSI1",
        KeyConditionExpression="GSI1PK = :gsi1pk",
        ExpressionAttributeValues={
            ":gsi1pk": {"S": "REST#{}".format(restaurant_name)},
        },
        ScanIndexForward=False,
        Limit=6,
    )

    restaurant = Restaurant(resp["Items"][0])
    restaurant.reviews = [Review(item) for item in resp["Items"][1:]]

    return restaurant


restaurant = fetch_restaurant_summary("AnyCompany Fine Dining")

print(restaurant)
for review in restaurant.reviews:
    print(review)

Overwriting application/fetch_restaurant_summary.py


설정은 앞의 두 예제와 유사합니다. 필요한 라이브러리를 가져온 후에는 애플리케이션의 함수와 유사한 **fetch_restaurant_summary** 함수가 있습니다. 이 함수는 레스토랑 이름을 가져와서 **Restaurant** 항목과 해당 레스토랑의 가장 최근 **Review** 항목 5개를 반환합니다.

이 함수는 **쿼리** 작업을 사용하여 이 작업을 수행합니다. DynamoDB에서 **쿼리**를 사용할 때는 검색하려는 항목을 설명하는 [키 조건 표현식](https://docs.aws.amazon.com/ko_kr/amazondynamodb/latest/developerguide/Query.html#Query.KeyConditionExpressions)을 지정해야 합니다. 키 조건 표현식에는 원하는 파티션 키와 정확히 일치하는 항목이 포함되어야 하며 일치시키려는 정렬 키에 대한 조건이 포함될 수 있습니다.

이 **쿼리** 작업에서는 **GSI1** 보조 인덱스에 대해 작업을 실행합니다. 특정 레스토랑에 대한 데이터가 들어 있는 정확한 파티션 키를 지정합니다. 또한 **ScanIndexForward=False** 속성을 지정합니다. 이는 항목 컬렉션에서 항목을 역순으로 읽으려는 것을 나타냅니다. 이렇게 하면 항목 컬렉션의 마지막 항목인 **Restaurant** 항목부터 시작하여 컬렉션의 가장 최근 **Review** 항목을 읽습니다. 마지막으로 6개 항목으로 제한되어 있으므로 **Restaurant** 항목과 가장 최근의 **Review** 항목 5개만 검색할 수 있습니다.

파일 맨 아래에는 레스토랑 이름이 **AnyCompany Fine Dining**인 함수를 호출하는 예가 나와 있습니다. 다음 명령을 실행하여 스크립트를 실행합니다.

In [17]:
!python application/fetch_restaurant_summary.py

Restaurant<AnyCompany Fine Dining -- Fine Dining>
Review<AnyCompany Fine Dining -- markmartin (2020-05-24T11:59:44)>
Review<AnyCompany Fine Dining -- kgraham (2020-05-14T15:01:52)>
Review<AnyCompany Fine Dining -- ewilliams (2020-05-13T02:36:30)>
Review<AnyCompany Fine Dining -- hannah21 (2020-05-04T03:44:26)>
Review<AnyCompany Fine Dining -- john97 (2020-04-27T20:45:52)>


다음과 같은 출력이 표시될 것입니다.

<img src="img/p9.png" width="600" height="400">

출력에는 **Restaurant** 항목과 **AnyCompany Fine Dining**에 대한 가장 최근의 **Review** 항목 5개가 인쇄되었습니다.

이 모듈에서는 애플리케이션 코드에서 몇 가지 액세스 패턴을 구현하는 방법을 배웠습니다. 먼저 조건 표현식을 사용하여 기존 항목을 덮어쓰지 않도록 하는 방법을 살펴보았습니다. 그런 다음 DynamoDB 트랜잭션을 사용하여 단일 요청에서 여러 항목에 대해 작업했습니다. 마지막으로 쿼리 작업을 사용하여 단일 요청에서 여러 이질적인 항목 유형을 검색했습니다.

다음 모듈에서는 이 단원에서 만든 리소스를 정리하는 방법을 살펴봅니다.

### 5. 리소스 정리하기

이 단원에서는 애플리케이션에서 레스토랑 평가 서비스의 기본 데이터베이스로 사용할 DynamoDB 테이블을 만들었습니다. DynamoDB는 대부분의 OLTP 애플리케이션에 적합하며, 특히 대규모 애플리케이션과 서버리스 애플리케이션에서 인기가 높습니다. DynamoDB는 확장 시 빠르고 일관된 성능을 제공할 뿐만 아니라 유연한 빌링 모델과 핸즈오프 운영 모델을 제공합니다.

이 모듈에서는 추가 요금이 발생하지 않도록 이 단원에서 만든 리소스를 정리합니다.

먼저 DynamoDB 테이블을 삭제합니다. **scripts**/ 디렉토리에 **delete_table.py**라는 스크립트가 있습니다. 여기에는 Boto3를 사용하여 테이블을 삭제하는 Python 스크립트가 포함되어 있습니다.

스크립트를 실행하고 테이블을 삭제하려면 터미널에서 다음 명령을 실행합니다.

In [18]:
!python scripts/delete_table.py

Table deleted successfully.


테이블이 성공적으로 삭제되었음을 나타내는 출력이 표시되어야 합니다.